In [ ]:
!pip3 install torch torchvision
!pip3 install pillow==4.1.1

In [ ]:
%matplotlib inline
from PIL import Image

import requests
from io import BytesIO

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt


import torchvision.transforms as transforms
import torchvision.models as models

import copy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class ContentLoss(nn.Module):

        def __init__(self, target,):
            super(ContentLoss, self).__init__()
            # we 'detach' the target content from the tree used
            # to dynamically compute the gradient: this is a stated value,
            # not a variable. Otherwise the forward method of the criterion
            # will throw an error.
            self.target = target.detach()#это константа. Убираем ее из дерева вычеслений
            self.loss = F.mse_loss(self.target, self.target )#to initialize with something

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input

In [ ]:
class StyleLoss(nn.Module):
        def gram_matrix(self,input):
            batch_size , h, w, f_map_num = input.size()  # batch size(=1)
            # b=number of feature maps
            # (h,w)=dimensions of a feature map (N=h*w)

            features = input.view(batch_size * h, w * f_map_num)  # resise F_XL into \hat F_XL

            G = torch.mm(features, features.t())  # compute the gram product

            # we 'normalize' the values of the gram matrix
            # by dividing by the number of element in each feature maps.
            return G.div(batch_size * h * w * f_map_num)
    
        def __init__(self, target_feature):
            super(StyleLoss, self).__init__()
            self.target = self.gram_matrix(target_feature).detach()
            self.loss = F.mse_loss(self.target, self.target)# to initialize with something

        def forward(self, input):
            G = self.gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input

In [ ]:
class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            # .view the mean and std to make them [C x 1 x 1] so that they can
            # directly work with image Tensor of shape [B x C x H x W].
            # B is batch size. C is number of channels. H is height and W is width.
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            # normalize img
            return (img - self.mean) / self.std

In [ ]:
class ganam_style(object):
    
    def image_loader(self,image_name):
        imsize = 256  

        loader = transforms.Compose([
        transforms.Resize(imsize),  # нормируем размер изображения
        transforms.CenterCrop(imsize),
        transforms.ToTensor()])  # превращаем в удобный формат
        
        image = Image.open(BytesIO(requests.get(image_name).content))
        image = loader(image).unsqueeze(0)
        return image.to(device, torch.float)

    def __init__(self, content_img, style_img = "drive/My Drive/Телеграм бот/images/picasso.jpg"):
        self.style_img = self.image_loader(style_img)
        self.content_img = self.image_loader(content_img)
        
    def get_style_model_and_losses(self,normalization_mean, normalization_std,\
                                       style_img, content_img,cnn = models.vgg19(pretrained=True).features.to(device).eval(),\
                                       content_layers=['conv_4'],
                                       style_layers=['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']):
            cnn = copy.deepcopy(cnn)

            # normalization module
            normalization = Normalization(normalization_mean, normalization_std).to(device)

            # just in order to have an iterable access to or list of content/syle
            # losses
            content_losses = []
            style_losses = []

            # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
            # to put in modules that are supposed to be activated sequentially
            model = nn.Sequential(normalization)

            i = 0  # increment every time we see a conv
            for layer in cnn.children():
                if isinstance(layer, nn.Conv2d):
                    i += 1
                    name = 'conv_{}'.format(i)
                elif isinstance(layer, nn.ReLU):
                    name = 'relu_{}'.format(i)
                    # The in-place version doesn't play very nicely with the ContentLoss
                    # and StyleLoss we insert below. So we replace with out-of-place
                    # ones here.
                    #Переопределим relu уровень
                    layer = nn.ReLU(inplace=False)
                elif isinstance(layer, nn.MaxPool2d):
                    name = 'pool_{}'.format(i)
                elif isinstance(layer, nn.BatchNorm2d):
                    name = 'bn_{}'.format(i)
                else:
                    raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

                model.add_module(name, layer)

                if name in content_layers:
                    # add content loss:
                    target = model(content_img).detach()
                    content_loss = ContentLoss(target)
                    model.add_module("content_loss_{}".format(i), content_loss)
                    content_losses.append(content_loss)

                if name in style_layers:
                    # add style loss:
                    target_feature = model(style_img).detach()
                    style_loss = StyleLoss(target_feature)
                    model.add_module("style_loss_{}".format(i), style_loss)
                    style_losses.append(style_loss)

            # now we trim off the layers after the last content and style losses
            #выбрасываем все уровни после последенего styel loss или content loss
            for i in range(len(model) - 1, -1, -1):
                if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                    break

            model = model[:(i + 1)]

            return model, style_losses, content_losses    
    
    def get_input_optimizer(self,input_img):
            # this line to show that input is a parameter that requires a gradient
            #добоваляет содержимое тензора катринки в список изменяемых оптимизатором параметров
            optimizer = optim.LBFGS([input_img.requires_grad_()]) 
            return optimizer

    def run_style_transfer(self,input_img,content_img,style_img,  cnn = models.vgg19(pretrained=True).features.to(device).eval(),\
                           normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device),\
                           normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device),\
                            num_steps=500,
                            style_weight=100000, content_weight=1):
            """Run the style transfer."""
            # print('Building the style transfer model..')
            model, style_losses, content_losses = self.get_style_model_and_losses(\
                normalization_mean = normalization_mean, normalization_std = normalization_std, style_img = style_img, content_img = content_img,cnn = cnn)
            optimizer = self.get_input_optimizer(input_img)

            # print('Optimizing..')
            run = [0]
            while run[0] <= num_steps:

                def closure():
                    # correct the values 
                    # это для того, чтобы значения тензора картинки не выходили за пределы [0;1]
                    input_img.data.clamp_(0, 1)

                    optimizer.zero_grad()

                    model(input_img)

                    style_score = 0
                    content_score = 0

                    for sl in style_losses:
                        style_score += sl.loss
                    for cl in content_losses:
                        content_score += cl.loss

                    #взвешивание ощибки
                    style_score *= style_weight
                    content_score *= content_weight

                    loss = style_score + content_score
                    loss.backward()

                    run[0] += 1
                    # if run[0] % 50 == 0:
                        # print("run {}:".format(run))
                        # print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                            # style_score.item(), content_score.item()))
                        # print()

                    return style_score + content_score

                optimizer.step(closure)

            # a last correction...
            input_img.data.clamp_(0, 1)

            return input_img  

    def imshow(self,tensor, title=None):
        unloader = transforms.ToPILImage()
        # plt.ion() 
        image = tensor.cpu().clone()   
        image = image.squeeze(0)      # функция для отрисовки изображения
        image = unloader(image)
        buf = BytesIO()
        image.save(buf, format='JPEG',)
        byte_im = buf.getvalue()
        return byte_im
        # plt.imshow(image)
        # if title is not None:
        #     plt.title(title)
        # plt.pause(0.001) 
        
    def styling(self):
        output = self.run_style_transfer(input_img = self.content_img.clone(),content_img = self.content_img, style_img = self.style_img)
        return self.imshow(output)
        # return output.type(torch.uint8)
        # plt.figure()
        #plt.imsave(output, 'output.png')
        # sphinx_gallery_thumbnail_number = 4
        # plt.ioff()
        # plt.show()

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth


In [ ]:
pip install pyTelegramBotAPI

     |████████████████████████████████| 71kB 6.4MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.1-cp36-none-any.whl size=50883 sha256=4f6a6269072d22a2fd3c5022e1bfb1454dd0b14e69eedc999f17a028637c5c20
  Stored in directory: /root/.cache/pip/wheels/cd/b2/2c/eac6af3343b21f907123ce013d20ad5ad70c2c3731072d98bf
Successfully built pyTelegramBotAPI


In [ ]:
import telebot
from telebot import types

token = "ваш токен"
bot = telebot.TeleBot(token)
url = "https://api.telegram.org/file/bot" + token + "/"

content_path = None
style_path = None
wait_content = None
wait_style = None
step = None

@bot.message_handler(content_types = ['photo'])
def handle_photo(message):
    global step, content_path,style_path,wait_content,wait_style
    # print(step, content_path,style_path,wait_content,wait_style)
    if step == 1:   
      if wait_content == 1 and wait_style is None:
        content_path = bot.get_file(message.photo[len(message.photo)-1].file_id).file_path
        bot.send_message(message.chat.id,'Отлично! Жду картинку стиля')
        step = 2
      elif wait_content is None and wait_style == 1:
        style_path = bot.get_file(message.photo[len(message.photo)-1].file_id).file_path
        bot.send_message(message.chat.id,'Отлично! Жду картинку контента')
        step = 2

    elif step == 2:   
      if content_path is not None:
        style_path = bot.get_file(message.photo[len(message.photo)-1].file_id).file_path
        bot.send_message(message.chat.id,'Секундочку...')
        a = ganam_style(content_img = url+ content_path, style_img = url + style_path)
        bot.send_photo(message.chat.id,a.styling())
        bot.send_message(message.chat.id,'Вуаля!')
        step, content_path,style_path,wait_content,wait_style = None,None,None,None,None
      else: 
        content_path = bot.get_file(message.photo[len(message.photo)-1].file_id).file_path
        bot.send_message(message.chat.id,'Секундочку...')
        a = ganam_style(content_img = url+ content_path, style_img = url + style_path)
        bot.send_photo(message.chat.id,a.styling())
        bot.send_message(message.chat.id,'Вуаля!')
        step, content_path,style_path,wait_content,wait_style = None,None,None,None,None
    else:
        bot.send_message(message.from_user.id, 'Давай для начала определимся, что ты хочешь получить, напиши /start или /help')

@bot.message_handler(commands = ['start','help'])
def send_welcome(message):
    global step, content_path,style_path,wait_content,wait_style
    step, content_path,style_path,wait_content,wait_style = None,None,None,None,None
    keyboard = types.InlineKeyboardMarkup()
    key_content = types.InlineKeyboardButton(text='Content picture', callback_data='content')
    keyboard.add(key_content)
    key_style= types.InlineKeyboardButton(text='Style picture', callback_data='style')
    keyboard.add(key_style)
    bot.send_message(message.from_user.id, text = "Приветствую! Взгляни на возможности Style Transfer! Для Style Transfer процедуры необходимы две картинки: картинка, которую будем преобразовывать (content picture)\
 и картинка, стиль которой будем использовать для преобразования первой картинки (style picture). Выбери, какую картинку хочешь загрузить первой",reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    global step, content_path,style_path,wait_content,wait_style   
    if call.data == 'content':
      bot.send_message(call.message.chat.id,'Жду от тебя картинку с контентом')
      step = 1
      wait_content = 1
    elif call.data == 'style':
      bot.send_message(call.message.chat.id,'Жду от тебя картинку стиля')
      step = 1
      wait_style = 1

       
@bot.message_handler(content_types=['text','audio','document'])
def check_sending(message):
      if step is None:
        bot.send_message(message.from_user.id, 'Для активации бота напиши /start или /help')
      else:
        bot.send_message(message.from_user.id, 'Не, не, не, пришли картинку')

bot.polling(none_stop=True, interval=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
